# Pivot_Longer : One function to cover transformations from wide to long form.

In [1]:
import janitor
import pandas as pd
import numpy as np

Unpivoting(reshaping data from wide to long form) in Pandas is executed either through [pd.melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html), [pd.wide_to_long](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.wide_to_long.html), or [pd.stack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). However, there are scenarios where a few more steps are required to massage the data into the long form we desire. Take the dataframe below, copied from [Stack Overflow](https://stackoverflow.com/questions/64061588/pandas-melt-multiple-columns-to-tabulate-a-dataset#64062002): 

In [2]:
df = pd.DataFrame(
        {
            "id": [1, 2, 3],
            "M_start_date_1": [201709, 201709, 201709],
            "M_end_date_1": [201905, 201905, 201905],
            "M_start_date_2": [202004, 202004, 202004],
            "M_end_date_2": [202005, 202005, 202005],
            "F_start_date_1": [201803, 201803, 201803],
            "F_end_date_1": [201904, 201904, 201904],
            "F_start_date_2": [201912, 201912, 201912],
            "F_end_date_2": [202007, 202007, 202007],
        }
    )

df

,id,M_start_date_1,M_end_date_1,M_start_date_2,M_end_date_2,F_start_date_1,F_end_date_1,F_start_date_2,F_end_date_2
0,1,201709,201905,202004,202005,201803,201904,201912,202007
1,2,201709,201905,202004,202005,201803,201904,201912,202007
2,3,201709,201905,202004,202005,201803,201904,201912,202007


A [beautiful solution](https://stackoverflow.com/a/64062027/7175713) was proferred : 

In [3]:
df1 = df.set_index('id')
df1.columns = df1.columns.str.split('_', expand=True)
df1 = (df1.stack(level=[0,2,3])
          .sort_index(level=[0,1], ascending=[True, False])
          .reset_index(level=[2,3], drop=True)
          .sort_index(axis=1, ascending=False)
          .rename_axis(['id','cod'])
          .reset_index())

df1

,id,cod,start,end
0,1,M,201709,201905
1,1,M,202004,202005
2,1,F,201803,201904
3,1,F,201912,202007
4,2,M,201709,201905
5,2,M,202004,202005
6,2,F,201803,201904
7,2,F,201912,202007
8,3,M,201709,201905
9,3,M,202004,202005


We propose an alternative, based on [pandas melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html), that abstracts the reshaping mechanism, allows the user to focus on the task, can be applied to other scenarios,  and is chainable : 

In [4]:
df.pivot_longer("id", names_to= ("cod", ".value"), names_pattern="(M|F)_(start|end)_.+")

,id,cod,start,end
0,1,M,201709,201905
1,1,M,202004,202005
2,1,F,201803,201904
3,1,F,201912,202007
4,2,M,201709,201905
5,2,M,202004,202005
6,2,F,201803,201904
7,2,F,201912,202007
8,3,M,201709,201905
9,3,M,202004,202005


`pivot_longer` is not a new idea; we unashamedly stole it from R's [tidyr](https://tidyr.tidyverse.org/reference/pivot_longer.html). Do note that the `pivot_longer` function is designed primarily to work with single indexed dataframes; for MultiIndex dataframes, `pandas_melt` is more than adequate. 

`pivot_longer` can melt dataframes easily; it is just a wrapper around `pd.melt` : 

In [5]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})

df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [6]:
df.pivot_longer(index='A', column_names=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


You can dynamically select columns, using regular expressions with the `janitor.patterns` function (inspired by R's data.table's [patterns](https://rdatatable.gitlab.io/data.table/reference/patterns.html) function, and is really just a wrapper around `re.compile`), especially if it is a lot of column names, and you are *lazy* like me  😄

In [7]:
url = 'https://github.com/tidyverse/tidyr/raw/master/data-raw/billboard.csv'
df = pd.read_csv(url)

df

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,86,83.0,77.0,74.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,85,83.0,83.0,82.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,95,94.0,91.0,85.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,99,99.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# unpivot all columns that start with 'wk'
df.pivot_longer(column_names = janitor.patterns("^(wk)"), names_to='week')

,artist,date.entered,time,track,year,week,value
0,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk1,87.0
1,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk2,82.0
2,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk3,72.0
3,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk4,77.0
4,2 Pac,2000-02-26,4:22,Baby Don't Cry (Keep...,2000,wk5,87.0
...,...,...,...,...,...,...,...
24087,matchbox twenty,2000-04-29,4:12,Bent,2000,wk72,NaN
24088,matchbox twenty,2000-04-29,4:12,Bent,2000,wk73,NaN
24089,matchbox twenty,2000-04-29,4:12,Bent,2000,wk74,NaN
24090,matchbox twenty,2000-04-29,4:12,Bent,2000,wk75,NaN


`pivot_longer` can also unpivot paired columns. Let's look at some examples from pandas `wide_to_long` docs : 

In [9]:
df = pd.DataFrame({
    'famid': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'birth': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'ht1': [2.8, 2.9, 2.2, 2, 1.8, 1.9, 2.2, 2.3, 2.1],
    'ht2': [3.4, 3.8, 2.9, 3.2, 2.8, 2.4, 3.3, 3.4, 2.9]
})

df

,famid,birth,ht1,ht2
0,1,1,2.8,3.4
1,1,2,2.9,3.8
2,1,3,2.2,2.9
3,2,1,2.0,3.2
4,2,2,1.8,2.8
5,2,3,1.9,2.4
6,3,1,2.2,3.3
7,3,2,2.3,3.4
8,3,3,2.1,2.9


In the data above, the `height` is paired with `age`. Let's see how `pivot_longer` handles this:

In [10]:
df.pivot_longer(index=['famid','birth'], names_to=('.value', 'age'), names_pattern=r"(ht)(\d)")

,famid,birth,age,ht
0,1,1,1,2.8
1,1,1,2,3.4
2,1,2,1,2.9
3,1,2,2,3.8
4,1,3,1,2.2
5,1,3,2,2.9
6,2,1,1,2.0
7,2,1,2,3.2
8,2,2,1,1.8
9,2,2,2,2.8


The first step in unpivoting a dataframe with a paired column is to specify the part of the column that will become a header, and the other part that becomes unpivoted. The `.value` symbol helps us achieve that. It indicates to the `pivot_longer` function that `ht` will be the new column name, while the rest of the data in the column(the numbers) will be unpivoted under the `age` column. How does `pivot_longer` associate `.value` to the part of the column name? The `names_pattern` or the `names_sep` arguments. For this, the `names_pattern` argument, which accepts regular expression, is a better fit; `pivot_longer` takes care of the rest.

`pd.wide_to_long` handles this already, so why bother? Let's look at another scenario where `pd.wide_to_long` would need a few more steps : 

In [11]:
#https://community.rstudio.com/t/pivot-longer-on-multiple-column-sets-pairs/43958
df = pd.DataFrame(
    {
        "off_loc": ["A", "B", "C", "D", "E", "F"],
        "pt_loc": ["G", "H", "I", "J", "K", "L"],
        "pt_lat": [
            100.07548220000001,
            75.191326,
            122.65134479999999,
            124.13553329999999,
            124.13553329999999,
            124.01028909999998,
        ],
        "off_lat": [
            121.271083,
            75.93845266,
            135.043791,
            134.51128400000002,
            134.484374,
            137.962195,
        ],
        "pt_long": [
            4.472089953,
            -144.387785,
            -40.45611048,
            -46.07156181,
            -46.07156181,
            -46.01594293,
        ],
        "off_long": [
            -7.188632000000001,
            -143.2288569,
            21.242563,
            40.937416999999996,
            40.78472,
            22.905889000000002,
        ],
    }
)

df

,off_loc,pt_loc,pt_lat,off_lat,pt_long,off_long
0,A,G,100.075482,121.271083,4.472090,-7.188632
1,B,H,75.191326,75.938453,-144.387785,-143.228857
2,C,I,122.651345,135.043791,-40.456110,21.242563
3,D,J,124.135533,134.511284,-46.071562,40.937417
4,E,K,124.135533,134.484374,-46.071562,40.784720
5,F,L,124.010289,137.962195,-46.015943,22.905889


We can unpivot with `pd.wide_to_long` by first reorganising the columns : 

In [12]:
df1 = df.copy()
df1.columns = ["_".join(col.split("_")[::-1]) for col in df1.columns]
df1.columns

Index(['loc_off', 'loc_pt', 'lat_pt', 'lat_off', 'long_pt', 'long_off'], dtype='object')

Nowe we can unpivot : 

In [13]:
pd.wide_to_long(
    df1.reset_index(),
    stubnames=["loc", "lat", "long"],
    sep="_",
    i="index",
    j="set",
    suffix="\w+",
)

loc         lat        long
index set                            
0     off   A  121.271083   -7.188632
      pt    G  100.075482    4.472090
1     off   B   75.938453 -143.228857
      pt    H   75.191326 -144.387785
2     off   C  135.043791   21.242563
      pt    I  122.651345  -40.456110
3     off   D  134.511284   40.937417
      pt    J  124.135533  -46.071562
4     off   E  134.484374   40.784720
      pt    K  124.135533  -46.071562
5     off   F  137.962195   22.905889
      pt    L  124.010289  -46.015943

Notice that we had to reset the dataframe to get an index variable. We can abstract all this away, using `pivot_longer` :

In [14]:
df.pivot_longer(
            names_to=["set", ".value"], names_pattern="(.+)_(.+)"
        )

,set,loc,lat,long
0,off,A,121.271083,-7.188632
1,pt,G,100.075482,4.472090
2,off,B,75.938453,-143.228857
3,pt,H,75.191326,-144.387785
4,off,C,135.043791,21.242563
5,pt,I,122.651345,-40.456110
6,off,D,134.511284,40.937417
7,pt,J,124.135533,-46.071562
8,off,E,134.484374,40.784720
9,pt,K,124.135533,-46.071562


In [15]:
# A way to see the pairings, to see what is linked to `.value`, which `pivot_longer` uses in reshaping the data.

# names_to =     ["set", ".value"]
# names_pattern = "(.+)_(.+)"
# column _names =   off_loc
#                   off_lat
#                   off_long

Again, the key here is the `.value` symbol. `names_to` is set as ('set', '.value'), and the `names_pattern` as "(.+)_(.+)". This tells `pivot_longer` to get the `loc`, `lat`, and `long` out of the existing column names, make them new column names, while unpivoting `off` and `pt` to a new column with the header `set`. Notice that we did not have to reset the index;  `pivot_longer` allows you to focus on what you want, so you can get it and move on. 

Let's look at some more examples : 

In [16]:
 # https://stackoverflow.com/questions/45123924/convert-pandas-dataframe-from-wide-to-long/45124130
 df = pd.DataFrame([{'a_1': 2, 'ab_1': 3, 'ac_1': 4, 'a_2': 5, 'ab_2': 6, 'ac_2': 7}]
)
df

,a_1,ab_1,ac_1,a_2,ab_2,ac_2
0,2,3,4,5,6,7


The data above requires extracting `a`, `ab` and `ac` from `1` and `2`. This is another example of a paired column. We could solve this using `pd.wide_to_long`; infact there is a very good solution from [Stack Overflow](https://stackoverflow.com/a/45124775/7175713)

In [17]:
df1 = df.copy()
df1['id'] = df1.index
pd.wide_to_long(df1, ['a','ab','ac'],i='id',j='num',sep='_')

a  ab  ac
id num           
0  1    2   3   4
   2    5   6   7

Or you could simply pass the buck to `pivot_longer` : 

In [18]:
df.pivot_longer(names_to=('.value','num'), names_sep='_')

,num,a,ab,ac
0,1,2,3,4
1,2,5,6,7


In the solution above, we used the `names_sep`, as it is more convenient. A few more examples to get you familiar with the `.value` symbol.

In [19]:
# https://stackoverflow.com/questions/55403008/pandas-partial-melt-or-group-melt
df = pd.DataFrame([[1,1,2,3,4,5,6],[2,7,8,9,10,11,12]], 
                      columns=['id', 'ax','ay','az','bx','by','bz'])

df

,id,ax,ay,az,bx,by,bz
0,1,1,2,3,4,5,6
1,2,7,8,9,10,11,12


In [20]:
df.pivot_longer(index='id', names_to=('name','.value'), names_pattern='(.)(.)')

,id,name,x,y,z
0,1,a,1,2,3
1,1,b,4,5,6
2,2,a,7,8,9
3,2,b,10,11,12


For the code above `.value` is paired with `x`, `y`, `z`, while `a`, `b`, and `c` are unpivoted into the `name` column. 

In the dataframe below, we need to unpivot the data, keeping only the suffix `hi`, and pulling out the number between `A` and `g`

In [21]:
# https://stackoverflow.com/questions/35929985/melt-a-data-table-with-a-column-pattern
df = pd.DataFrame([{'id': 1, 'A1g_hi': 2, 'A2g_hi': 3, 'A3g_hi': 4, 'A4g_hi': 5}])
df

,id,A1g_hi,A2g_hi,A3g_hi,A4g_hi
0,1,2,3,4,5


In [22]:
df.pivot_longer('id', names_to=['time','.value'], names_pattern="A(\d)g_(hi)")

,id,time,hi
0,1,1,2
1,1,2,3
2,1,3,4
3,1,4,5


One more example on the `.value` symbol for paired columns : 

In [23]:
# https://stackoverflow.com/questions/59477686/python-pandas-melt-single-column-into-two-seperate
df = pd.DataFrame({'id': [1, 2], 'A_value': [50, 33], 'D_value': [60, 45]})
df

,id,A_value,D_value
0,1,50,60
1,2,33,45


In [24]:
df.pivot_longer('id', names_to=('value_type', '.value'), names_sep='_')

,id,value_type,value
0,1,A,50
1,1,D,60
2,2,A,33
3,2,D,45


There are scenarios where we need to unpivot the data, but only keep some specific names in the unpivoted data. Let's see an example below:

In [25]:
# https://stackoverflow.com/questions/59550804/melt-column-by-substring-of-the-columns-name-in-pandas-python
df = pd.DataFrame({'subject': [1, 2],
 'A_target_word_gd': [1, 11],
 'A_target_word_fd': [2, 12],
 'B_target_word_gd': [3, 13],
 'B_target_word_fd': [4, 14],
 'subject_type': ['mild', 'moderate']})

df

,subject,A_target_word_gd,A_target_word_fd,B_target_word_gd,B_target_word_fd,subject_type
0,1,1,2,3,4,mild
1,2,11,12,13,14,moderate


In the dataframe above, `A` and `B` represents conditions, while the suffixes `gd` and `fd` represent value types. We are not interested in the words in the middle (`_target_word`). We could solve it this way (solution copied from [Stack Overflow](https://stackoverflow.com/a/59550967/7175713)) : 

In [26]:
new_df =pd.melt(df, id_vars=['subject_type','subject'], var_name='abc').sort_values(by=['subject', 'subject_type'])
new_df['cond']=new_df['abc'].apply(lambda x: (x.split('_'))[0])
new_df['value_type']=new_df.pop('abc').apply(lambda x: (x.split('_'))[-1])
new_df

,subject_type,subject,value,cond,value_type
0,mild,1,1,A,gd
2,mild,1,2,A,fd
4,mild,1,3,B,gd
6,mild,1,4,B,fd
1,moderate,2,11,A,gd
3,moderate,2,12,A,fd
5,moderate,2,13,B,gd
7,moderate,2,14,B,fd


Or, we could just pass the buck to `pivot_longer` : 

In [27]:
df.pivot_longer(index=['subject', 'subject_type'], names_to=('cond','value_type'), names_pattern='([A-Z]).*(gd|fd)')

,subject,subject_type,cond,value_type,value
0,1,mild,A,gd,1
1,1,mild,A,fd,2
2,1,mild,B,gd,3
3,1,mild,B,fd,4
4,2,moderate,A,gd,11
5,2,moderate,A,fd,12
6,2,moderate,B,gd,13
7,2,moderate,B,fd,14


In the above, we pass in the new names of the columns to `names_to`('cond', 'value_type'), and pass the groups to be extracted as a regular expression to `names_pattern`. 

Here's another example where `pivot_longer` abstracts the process and makes reshaping easier.

In the dataframe below, we would like to unpivot the data and separate the column names into individual columns(`vault` should be an `event` column, `2012` should be a `year` column and `f` should be a `gender` column)

In [28]:
# https://dcl-wrangle.stanford.edu/pivot-advanced.html
df = pd.DataFrame(
            {
                "country": ["United States", "Russia", "China"],
                "vault_2012_f": [
                    48.132,
                    46.36600000000001,
                    44.266000000000005,
                ],
                "vault_2012_m": [46.632, 46.86600000000001, 48.316],
                "vault_2016_f": [
                    46.86600000000001,
                    45.733000000000004,
                    44.332,
                ],
                "vault_2016_m": [45.865, 46.033, 45.0],
                "floor_2012_f": [45.36600000000001, 41.599, 40.833],
                "floor_2012_m": [45.266000000000005, 45.308, 45.133],
                "floor_2016_f": [45.998999999999995, 42.032, 42.066],
                "floor_2016_m": [43.757, 44.766000000000005, 43.799],
            }
        )
df


,country,vault_2012_f,vault_2012_m,vault_2016_f,vault_2016_m,floor_2012_f,floor_2012_m,floor_2016_f,floor_2016_m
0,United States,48.132,46.632,46.866,45.865,45.366,45.266,45.999,43.757
1,Russia,46.366,46.866,45.733,46.033,41.599,45.308,42.032,44.766
2,China,44.266,48.316,44.332,45.000,40.833,45.133,42.066,43.799


We could achieve this with a combination of `pd.melt` and pandas string methods (or janitor's deconcatenate_columns method); or we could, again, pass the buck to `pivot_longer` : 

In [29]:
df.pivot_longer(index='country', names_to = ["event", "year", "gender"], names_sep="_", values_to='score')

,country,event,year,gender,score
0,United States,vault,2012,f,48.132
1,United States,vault,2012,m,46.632
2,United States,vault,2016,f,46.866
3,United States,vault,2016,m,45.865
4,United States,floor,2012,f,45.366
5,United States,floor,2012,m,45.266
6,United States,floor,2016,f,45.999
7,United States,floor,2016,m,43.757
8,Russia,vault,2012,f,46.366
9,Russia,vault,2012,m,46.866


`pivot_longer` does not solve all problems; no function does. Its aim is to act as a single point for transformations, on single indexed dataframes,  that require `pd.melt` or `pd.wide_to_long`, and offer more flexibility.